In [1]:
import pandas as pd
import utils

bios = pd.read_json("../../data/congress-bios.json")
currentBios = bios[bios.congresses.apply(lambda x: 118 in x)]

...what is that at 12395?

In [2]:
display(currentBios["profileText"][12395])

# darn it, Ben Sasse!

str12395 = (
    "a Senator from Nebraska; born in Nebraska City, Nebr., August 19, 1964; graduated"
    " Westside High School, Omaha, Nebr., 1982; B.A., biology, University of Chicago,"
    " 1986; M.B.A., marketing and finance, University of Chicago, 1991; online"
    " brokerage firm president and chief operating officer; co-owner of Chicago Cubs"
    " baseball team; unsuccessful candidate for the United States Senate in 2006;"
    " governor of Nebraska 2015-2023; appointed as a Republican to the United States"
    " Senate on January 12, 2023, to fill the vacancy caused by the resignation of"
    " Benjamin Sasse; took the oath of office on January 23, 2023."
)
currentBios.loc[12395, "profileText"] = str12395

'<!-- [if gte mso 9]><xml>\n <o:OfficeDocumentSettings>\n  <o:AllowPNG/>\n </o:OfficeDocumentSettings>\n</xml><![endif]--><!-- [if gte mso 9]><xml>\n <w:WordDocument>\n  <w:View>Normal</w:View>\n  <w:Zoom>0</w:Zoom>\n  <w:TrackMoves/>\n  <w:TrackFormatting/>\n  <w:PunctuationKerning/>\n  <w:ValidateAgainstSchemas/>\n  <w:SaveIfXMLInvalid>false</w:SaveIfXMLInvalid>\n  <w:IgnoreMixedContent>false</w:IgnoreMixedContent>\n  <w:AlwaysShowPlaceholderText>false</w:AlwaysShowPlaceholderText>\n  <w:DoNotPromoteQF/>\n  <w:LidThemeOther>EN-US</w:LidThemeOther>\n  <w:LidThemeAsian>X-NONE</w:LidThemeAsian>\n  <w:LidThemeComplexScript>X-NONE</w:LidThemeComplexScript>\n  <w:Compatibility>\n   <w:BreakWrappedTables/>\n   <w:SnapToGridInCell/>\n   <w:WrapTextWithPunct/>\n   <w:UseAsianBreakRules/>\n   <w:DontGrowAutofit/>\n   <w:SplitPgBreakAndParaMark/>\n   <w:EnableOpenTypeKerning/>\n   <w:DontFlipMirrorIndents/>\n   <w:OverrideTableStyleHps/>\n  </w:Compatibility>\n  <m:mathPr>\n   <m:mathFont m:val

In [3]:
currentBios = currentBios[
    ["givenName", "familyName", "congresses", "profileText", "jobPositions"]
]

In [4]:
currentBios.loc[:, "party"] = currentBios.jobPositions.apply(
    lambda x: x[-1]["congressAffiliation"]["partyAffiliation"][-1]["party"]["name"][0],
    # axis=1
)  # ["congress"]["congressNumber"]
currentBios.sample(5)

,givenName,familyName,congresses,profileText,jobPositions,party
1915,Jimmy,Gomez,"[115, 116, 117, 118]",a Representative from California; born in Full...,"[{'job': {'name': 'Representative', 'jobType':...",D
9596,Betty,McCollum,"[107, 108, 109, 110, 111, 112, 113, 114, 115, ...",A Representative from Minnesota; born in Minne...,"[{'job': {'name': 'Representative', 'jobType':...",D
12483,John,Thune,"[105, 106, 107, 109, 110, 111, 112, 113, 114, ...",A Senator and a Representative from South Dako...,"[{'job': {'name': 'Representative', 'jobType':...",R
12851,Charles,Schumer,"[97, 98, 99, 100, 101, 102, 103, 104, 105, 106...",A Senator and a Representative from New York; ...,"[{'job': {'name': 'Representative', 'jobType':...",D
8416,Marilyn,Strickland,"[117, 118]",a Representative from Washington; born in Seou...,"[{'job': {'name': 'Representative', 'jobType':...",D


In [5]:
def get_state_abbrev(text):
    for state in utils.state_list:
        if state.lower() in text.lower():
            return utils.state_dict[state]

    return None


currentBios.loc[:, "state"] = currentBios.profileText.apply(
    lambda x: get_state_abbrev(x.split(";")[0])
)

In [6]:
currentBios.sample(5)

,givenName,familyName,congresses,profileText,jobPositions,party,state
11727,Sheldon,Whitehouse,"[110, 111, 112, 113, 114, 115, 116, 117, 118]",A Senator from Rhode Island; born in Manhattan...,"[{'job': {'name': 'Senator', 'jobType': 'Congr...",D,RI
6205,Michael,Cloud,"[115, 116, 117, 118]",A Representative from Texas; born in Baton Rou...,"[{'job': {'name': 'Representative', 'jobType':...",R,TX
11240,Ro,Khanna,"[115, 116, 117, 118]",a Representative from California; born in Phil...,"[{'job': {'name': 'Representative', 'jobType':...",D,CA
9656,Theodore,Budd,"[115, 116, 117, 118]",a Representative from North Carolina; born in ...,"[{'job': {'name': 'Representative', 'jobType':...",R,NC
11197,Cory,Mills,[118],a Representative from Florida; born in Winter ...,"[{'job': {'name': 'Representative', 'jobType':...",R,FL


### Let's confirm that there are 2 senators from each State:

In [7]:
currentBios.loc[
    (
        currentBios.profileText.str.lower().str.contains("a senator from")
        | currentBios.profileText.str.lower().str.contains("a senator and a represent")
        | currentBios.profileText.str.lower().str.contains("a senator and represent")
    )
].state.value_counts().loc[lambda x: x != 2]

state
NE    3
CA    3
VT    1
OK    1
NC    1
Name: count, dtype: int64

Okay, I assume that the NE and CA anomalies are Ben Sasse retiring and Diane
Feinstein passing away, since the two of them were both members of the 118th
Congress. Let's check on that real quick:

In [8]:
currentBios.loc[
    (currentBios.familyName == "Sasse") | (currentBios.familyName == "Feinstein")
].index

Index([6720, 12640], dtype='int64')

I'm not going to drop either of them, since they were technically parts of the 118th Congress. But that still leaves the question of what's going on in VT, OK, and NC.

In [9]:
currentBios.loc[
    (
        currentBios.profileText.str.lower().str.contains("a senator from")
        | currentBios.profileText.str.lower().str.contains("a senator and a represent")
        | currentBios.profileText.str.lower().str.contains("a senator and represent")
    )
].loc[lambda x: x.state == "VT"]

,givenName,familyName,congresses,profileText,jobPositions,party,state
5280,Bernard,Sanders,"[102, 103, 104, 105, 106, 107, 108, 109, 110, ...",A Senator and a Representative from Vermont; b...,"[{'job': {'name': 'Representative', 'jobType':...",I,VT


Okay, looks like [Peter Welch](https://en.wikipedia.org/wiki/Peter_Welch) is missing...

In [10]:
currentBios.loc[lambda x: (x.givenName == "Peter") & (x.familyName == "Welch")]

,givenName,familyName,congresses,profileText,jobPositions,party,state
2107,Peter,Welch,"[110, 111, 112, 113, 114, 115, 116, 117, 118]",A Representative from Vermont; born in Springf...,"[{'job': {'name': 'Representative', 'jobType':...",D,VT


I guess they didn't update him in the Congressional Bioguide when he replaced Patrick Leahy? Luckily, this is an easy fix: I'll just do what I did with that massive misencoded html bio from earlier.

In [11]:
currentBios.loc[2107, "profileText"]

'A Representative from Vermont; born in Springfield, Hampden County, Mass., May 2, 1947; graduated from Cathedral High School, Springfield, Mass.; A.B., College of the Holy Cross, Worcester, Mass.; 1969; J.D., University of California, Berkeley, Calif., 1973; lawyer, private practice; member of the Vermont state senate, 1981-1989, 2002-2007, minority leader, 1983-1985, president pro tempore, 1985-1989, 2003-2007; unsuccessful candidate for nomination to the One Hundred First Congress in 1988; elected as a Democrat to the One Hundred Tenth and to the seven succeeding Congresses (January 3, 2007-January 3, 2023); was not a candidate for reelection to the House of Representatives but was elected as a Democrat to the United States Senate in 2022 for the term ending January 3, 2029.'

They even note that he was elected as a Democrat to the Senate at the end of his bio! If I were less lazy, I'd rewrite some code to use "elected as a x to the United States Senate," but I figure so many people get appointed that the way I'm doing it now is easier.

In [12]:
welchBio = (
    "A Senator and a Representative from Vermont; born in Springfield, Hampden County,"
    " Mass., May 2, 1947; graduated from Cathedral High School, Springfield, Mass.;"
    " A.B., College of the Holy Cross, Worcester, Mass.; 1969; J.D., University of"
    " California, Berkeley, Calif., 1973; lawyer, private practice; member of the"
    " Vermont state senate, 1981-1989, 2002-2007, minority leader, 1983-1985, president"
    " pro tempore, 1985-1989, 2003-2007; unsuccessful candidate for nomination to the"
    " One Hundred First Congress in 1988; elected as a Democrat to the One Hundred"
    " Tenth and to the seven succeeding Congresses (January 3, 2007-January 3, 2023);"
    " was not a candidate for reelection to the House of Representatives but was"
    " elected as a Democrat to the United States Senate in 2022 for the term ending"
    " January 3, 2029."
)
currentBios.loc[2107, "profileText"] = welchBio

In [13]:
currentBios.loc[
    (
        currentBios.profileText.str.lower().str.contains("a senator from")
        | currentBios.profileText.str.lower().str.contains("a senator and a represent")
        | currentBios.profileText.str.lower().str.contains("a senator and represent")
    )
].state.value_counts().loc[lambda x: x == 1]

state
OK    1
NC    1
Name: count, dtype: int64

Alright, that's VT down, now just Oklahoma and NC. My bet is that the same thing happened to Markwayne Mullin and Ted Budd.

In [14]:
currentBios.loc[
    (
        currentBios.profileText.str.lower().str.contains("a senator from")
        | currentBios.profileText.str.lower().str.contains("a senator and a represent")
        | currentBios.profileText.str.lower().str.contains("a senator and represent")
    )
].loc[lambda x: (x.state == "OK") | (x.state == "NC")]

,givenName,familyName,congresses,profileText,jobPositions,party,state
4740,James,Lankford,"[112, 113, 114, 115, 116, 117, 118]",<p>A Senator and a Representative from Oklahom...,"[{'job': {'name': 'Representative', 'jobType':...",R,OK
11543,Thomas,Tillis,"[114, 115, 116, 117, 118]",A Senator from North Carolina; born in Jackson...,"[{'job': {'name': 'Senator', 'jobType': 'Congr...",R,NC


Yep, my hunch was right.

In [15]:
display(
    currentBios.loc[lambda x: (x.givenName == "Markwayne") & (x.familyName == "Mullin")]
)
currentBios.loc[11757, "profileText"] = currentBios.loc[11757, "profileText"].replace(
    "a Representative from", "a Senator and a Representative from"
)

display(
    currentBios.loc[lambda x: (x.givenName == "Markwayne") & (x.familyName == "Mullin")]
)

,givenName,familyName,congresses,profileText,jobPositions,party,state
11757,Markwayne,Mullin,"[113, 114, 115, 116, 117, 118]","a Representative from Oklahoma; born in Tulsa,...","[{'job': {'name': 'Representative', 'jobType':...",R,OK


,givenName,familyName,congresses,profileText,jobPositions,party,state
11757,Markwayne,Mullin,"[113, 114, 115, 116, 117, 118]",a Senator and a Representative from Oklahoma; ...,"[{'job': {'name': 'Representative', 'jobType':...",R,OK


In [16]:
display(
    currentBios.loc[lambda x: (x.givenName == "Theodore") & (x.familyName == "Budd")]
)
currentBios.loc[9656, "profileText"] = currentBios.loc[9656, "profileText"].replace(
    "a Representative from", "a Senator and a Representative from"
)

display(
    currentBios.loc[lambda x: (x.givenName == "Theodore") & (x.familyName == "Budd")]
)

,givenName,familyName,congresses,profileText,jobPositions,party,state
9656,Theodore,Budd,"[115, 116, 117, 118]",a Representative from North Carolina; born in ...,"[{'job': {'name': 'Representative', 'jobType':...",R,NC


,givenName,familyName,congresses,profileText,jobPositions,party,state
9656,Theodore,Budd,"[115, 116, 117, 118]",a Senator and a Representative from North Caro...,"[{'job': {'name': 'Representative', 'jobType':...",R,NC


In [17]:
currentBios.loc[
    (
        currentBios.profileText.str.lower().str.contains("a senator from")
        | currentBios.profileText.str.lower().str.contains("a senator and a represent")
        | currentBios.profileText.str.lower().str.contains("a senator and represent")
    )
].state.value_counts().loc[lambda x: x == 1]

Series([], Name: count, dtype: int64)

Alright, now we can easily flag everyone who was a senator in the 118th congress! Let's make an indicator real quick:

In [18]:
currentBios["senator"] = (
    currentBios.profileText.str.lower().str.contains("a senator from")
    | currentBios.profileText.str.lower().str.contains("a senator and a represent")
    | currentBios.profileText.str.lower().str.contains("a senator and represent")
).astype(bool)

### Checking representatives

In [19]:
currentBios.senator.apply(lambda x: not x).sum()

np.int64(441)

Okay, this is what I expected! There are, after all, [6 non-voting members of the House of Representatives](https://en.wikipedia.org/wiki/Non-voting_members_of_the_United_States_House_of_Representatives), and 435 = 441 - 6. I don't really care about the non-voting members, though (no offense), so I'll be dropping them (as well as making sure each state has the correct number of reps).

I might want to write a more robust script to handle Congresses before 1960 (i.e. more robust "was this state a state at the time" handling), but that isn't a problem right now.

In [20]:
currentBios.loc[currentBios.senator.apply(lambda x: not x) & currentBios.state.isna()]

,givenName,familyName,congresses,profileText,jobPositions,party,state,senator
2306,Aumua,Radewagen,"[114, 115, 116, 117, 118]",a Delegate from American Samoa; born in Washin...,"[{'job': {'name': 'Delegate', 'jobType': 'Cong...",R,None,False
2586,Stacey,Plaskett,"[114, 115, 116, 117, 118]",A Delegate from the Virgin Islands; born in Ne...,"[{'job': {'name': 'Delegate', 'jobType': 'Cong...",D,None,False
3087,James,Moylan,[118],"a Delegate from Guam; born on July 18, 1962; g...","[{'job': {'name': 'Delegate', 'jobType': 'Cong...",R,None,False
3812,Jenniffer,González-Colón,"[115, 116, 117, 118]",a Resident Commissioner from Puerto Rico; born...,"[{'job': {'name': 'Resident Commissioner', 'jo...",R,None,False
5134,Gregorio,Sablan,"[111, 112, 113, 114, 115, 116, 117, 118]",a Delegate from the Northern Mariana Islands; ...,"[{'job': {'name': 'Delegate', 'jobType': 'Cong...",D,None,False
12811,Eleanor,Norton,"[102, 103, 104, 105, 106, 107, 108, 109, 110, ...",a Delegate from the District of Columbia; born...,"[{'job': {'name': 'Delegate', 'jobType': 'Cong...",D,None,False


In [21]:
currentBios.drop(
    currentBios.loc[
        currentBios.senator.apply(lambda x: not x) & currentBios.state.isna()
    ].index,
    inplace=True,
)

In [22]:
reps = {
    get_state_abbrev(k): v
    for k, v in zip(utils.number_of_reps.keys(), utils.number_of_reps.values())
}

In [23]:
# confirming that every state has the correct number of reps

[
    state
    for state in currentBios.loc[currentBios.senator.apply(lambda x: not x)]
    .state.value_counts()
    .index
    if currentBios.loc[currentBios.senator.apply(lambda x: not x)]
    .state.value_counts()
    .loc[state]
    != reps[state]
]

[]

In [24]:
currentBios["congressLabels"] = currentBios.apply(
    lambda row: f"{row.givenName} {row.familyName} ({row.party}-{row.state})",
    axis=1,
)

### Applying categories

Here, I put the csv into Google Sheets and did a whole lot more work on it — basically all manual, which wasn't fun. Turns out it's tough to hard-code which things ought to count as which categories. I tried some GPT-4 api calls, but I didn't end up doing too well. Good news is that, after poking around there, I can do it more efficiently here below.

The labels below are almost entirely the originals from the [New York Times piece](https://www.nytimes.com/interactive/2019/01/26/opinion/sunday/paths-to-congress.html) except that I added
- previously unsuccessful candidate for office,
- more specific master's degrees,
- previous experience in party politics.

I think I was also more generous with what counts as a medical degree, including nursing and osteopathic degrees.

You can see what I checked for each thing by examining the flags I've added for each category in in the `utils` module, though. 

In [25]:
def splitAndTest(profileText, contains, doesntContain=[]):
    bioItems = profileText.lower().split(";")
    if doesntContain:
        for bioItem in bioItems:
            for c in contains:
                if c.lower() in bioItem:
                    if not any(dc.lower() in bioItem for dc in doesntContain):
                        return bioItem
                    else:
                        continue
        return False

    else:
        for bioItem in bioItems:
            for c in contains:
                if c.lower() in bioItem:
                    return bioItem
        return False

In [26]:
currentBios["communityCollege"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x, utils.communityCollege, ["board", "professor", "instructor", "president"]
    )
)

In [27]:
currentBios["eliteCollege"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.eliteCollege,
        utils.otherMasters
        + utils.mba
        + utils.medDegree
        + utils.doctorate
        + utils.lawSchool
        + [
            "county",
            "post-graduate studies",
            "fellow",
            "high school",
            "medical school",
            "hospital",
            "texas southmost college",
            "extension school",
        ],
    )
)

In [28]:
currentBios["publicCollege"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.publicCollege,
        utils.otherMasters
        + utils.mba
        + utils.medDegree
        + utils.doctorate
        + utils.lawSchool
        + utils.education
        + [
            # "county",
            "post-graduate studies",
            "fellow",
            "high school",
            "medical school",
            "hospital",
            "texas southmost college",
            "extension school",
            "board of governors",
            "a.a.s.",
        ],
    )
)

In [29]:
currentBios["privateCollege"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.privateCollege,
        utils.otherMasters
        + utils.mba
        + utils.medDegree
        + utils.doctorate
        + utils.lawSchool
        + utils.education
        + [
            "county",
            "post-graduate studies",
            "fellow",
            "high school",
            "medical school",
            "hospital",
            "texas southmost college",
            "extension school",
            "board of governors",
            "a.a.s.",
        ],
    )
)

In [30]:
currentBios["noBA"] = currentBios.apply(
    lambda x: (
        (not x["communityCollege"])
        & (not x["eliteCollege"])
        & (not x["publicCollege"])
        & (not x["privateCollege"])
    ),
    axis=1,
)

In [31]:
currentBios["lawSchool"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.lawSchool)
)

In [32]:
currentBios["medDegree"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.medDegree)
)

In [33]:
currentBios["doctorate"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.doctorate)
)

In [34]:
currentBios["mba"] = currentBios.profileText.apply(lambda x: splitAndTest(x, utils.mba))

In [35]:
currentBios["otherMasters"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.otherMasters)
)

In [36]:
currentBios["military"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.military, ["academy", "college"])
)

In [37]:
currentBios["lawyerPrivatePractice"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.lawyerPrivatePractice)
)
currentBios["lawyerPrivatePractice"] = [
    (
        currentBios.lawyerPrivatePractice.iloc[i]
        if currentBios.lawyerPrivatePractice.iloc[i]
        else (
            "lawyer"
            if any(
                [
                    (
                        bioItem
                        if (bioItem.strip() == "lawyer")
                        | (bioItem.strip() == "attorney")
                        else False
                    )
                    for bioItem in currentBios.profileText.iloc[i].lower().split(";")
                ]
            )
            else False
        )
    )
    for i in range(len(currentBios))
]

In [38]:
currentBios["publicLawyerOrJudge"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x, utils.publicLawyerOrJudge, ["college counselor", "one of the managers"]
    )
)

In [39]:
currentBios["prevUnsuccessfulCandidate"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.prevUnsuccessfulCandidate)
)

In [40]:
currentBios["partyPolitics"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.partyPolitics + ["democratic", "republican"],
        [
            "elected as a republican",
            "united states",
            "unsuccessful",
            "independent",
            "select committee on the strategic competition",
            "governor of maine",
        ],
    )
)

In [41]:
currentBios["sciOrEng"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.sciOrEng, ["b.s.", "m.s."])
)

In [42]:
currentBios["religion"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.religion, ["b.s.", "m.s."])
)

In [43]:
currentBios["healthcare"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.healthcare, ["b.s.", "m.s.", "m.d."])
)

In [44]:
currentBios["blueCollarOrServiceJob"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.blueCollarOrServiceJob, ["academy"])
)

In [45]:
currentBios["education"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.education, ["elected as a republican"])
)

In [46]:
currentBios["nonprofitsAndUnions"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.nonprofitsAndUnions,
        [
            "credit union",
            "union, n.j.",
            "union county, n.j.",
            "union, miss.",
            "union city",
            "union theological seminary",
            "ph.d., union institute",
        ],
    )
)

In [47]:
currentBios["sports"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.sports, ["coachella valley high school"])
)

In [48]:
currentBios["media"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x, utils.media, ["anchorage", "authority", "authorized", "speechwriter"]
    )
)

In [49]:
currentBios["realEstate"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.realEstate)
)

In [50]:
currentBios["lawEnforcement"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.lawEnforcement, ["attorney"])
)

In [51]:
currentBios["farmingOrRanching"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x, utils.farmingOrRanching, ["chair, committee on agriculture", "administrator"]
    )
)

In [52]:
currentBios["businessOrManagement"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.businessOrManagement,
        [
            "campaign consultant",
            "central intelligence agency",
            "national nuclear security administration",
            "fairbanks",
            "committee on banking",
            "b.a., business administration",
            "county executive",
            "red bank, monmouth county",
            "executive office of the president",
            "committee on small business",
            "osceola",
            "nonprofit",
            "non-profit",
            "small business administration",
            "federal coordinator for gulf coast rebuilding",
            "m.b.a.",
            "m.p.a",
            "bankruptcy court",
            "business administration",
            "secretary of state",
            "government analyst",
            "california democratic state central executive committee",
            "executive director, human services",
            "jefferson county",
            "business regulation",
            "commission",
            "department of regulatory agencies",
            "nevada system of higher education",
            "republican state committee",
            "workers' compensation board",
            "afl-cio",
        ],
    )
)

In [53]:
currentBios["lobbyingOrPacs"] = currentBios.profileText.apply(
    lambda x: splitAndTest(x, utils.lobbyingOrActivism)
)

In [54]:
currentBios["localGov"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x, utils.localGov, ["graduated from", "b.a.", "unsuccessful candidate for"]
    )
)

In [55]:
currentBios["stateLeg"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.stateLeg,
        ["legislative assistant", "aide", "staff", "counsel", "fellow", "unsuccessful"],
    )
)

In [56]:
currentBios["electedFedOrStateOffice"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.electedFedOrStateOffice,
        [
            "appointed",
            "assistant u.s. secretary of state",
            "inland empire regional office",
            "counsel",
            "advisor",
            "unsuccessful",
            "lost general election",
        ],
    )
)

In [57]:
currentBios["nonElectedGovJob"] = currentBios.profileText.apply(
    lambda x: splitAndTest(
        x,
        utils.nonElectedGovJob + ["advisor"],
        [
            "county",
            "college",
            "development advisor",
            "emergency 9-1-1 dispatcher",
            "nonprofit organization",
            "n.y. council",
            "graduated",
            "postal inspector",
            "inland empire regional office",
            "chapel hill, n.c., police department",
            "long beach, calif",
        ],
    )
)

In [58]:
currentBios.to_json("../../data/118-processed-bios.json")